In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50    # steepness factor
num_blocks = 10   # number of expression blocks
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 32
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

## Heuristic - N1

In [7]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_dev, a_dev))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n1_50-10.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:13<00:00,  5.18it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean     85.916926               0.0      0.192038
std      32.425162               0.0      0.132293
min      16.546269               0.0      0.106474
25%      60.859942               0.0      0.138505
50%      79.301375               0.0      0.156199
75%     104.622241               0.0      0.201613
max     190.131414               0.0      2.859052
Number of infeasible solution: 0


## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 80   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2162.75
Epoch 1, Validation Loss: 357.60
Epoch 2, Validation Loss: 178.94
Epoch 3, Validation Loss: 192.99
Epoch 4, Validation Loss: 133.77
Epoch 5, Validation Loss: 124.76
Epoch 6, Validation Loss: 134.15
Epoch 7, Validation Loss: 104.76
Epoch 8, Validation Loss: 93.90
Epoch 9, Validation Loss: 88.85
Epoch 10, Validation Loss: 87.39
Epoch 11, Validation Loss: 81.44
Epoch 12, Validation Loss: 71.79
Epoch 13, Validation Loss: 87.41
Epoch 14, Validation Loss: 78.15
Epoch 15, Validation Loss: 68.68
Epoch 16, Validation Loss: 65.61
Epoch 17, Validation Loss: 59.36
Epoch 18, Validation Loss: 63.63
Epoch 19, Validation Loss: 70.51
Epoch 20, Validation Loss: 69.99
Epoch 21, Validation Loss: 61.57
Epoch 22, Validation Loss: 65.48
Epoch 23, Validation Loss: 63.70
Epoch 24, Validation Loss: 61.94
Epoch 25, Validation Loss: 93.32
Epoch 26, Validation Loss: 79.71
Epoch 27, Validation Loss: 61.77
Epoch 28, Validation Loss: 51.43
Epoch 29, Validation Loss: 63.30
Epoch 30, V

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_dev, a_dev))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lr_50-10.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 186.54it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean     49.402684               0.0      0.002846
std      30.019292               0.0      0.000890
min       7.369320               0.0      0.001000
25%      27.007854               0.0      0.002013
50%      41.792557               0.0      0.002662
75%      64.288207               0.0      0.003287
max     174.388602               0.0      0.008013
Number of infeasible solution: 0


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 80   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 1856.29
Epoch 1, Validation Loss: 293.59
Epoch 2, Validation Loss: 135.55
Epoch 3, Validation Loss: 91.44
Epoch 4, Validation Loss: 94.65
Epoch 5, Validation Loss: 71.15
Epoch 6, Validation Loss: 81.10
Epoch 7, Validation Loss: 71.31
Epoch 8, Validation Loss: 71.47
Epoch 9, Validation Loss: 76.20
Epoch 10, Validation Loss: 79.67
Epoch 11, Validation Loss: 71.66
Epoch 12, Validation Loss: 74.27
Epoch 13, Validation Loss: 69.22
Epoch 14, Validation Loss: 78.52
Epoch 15, Validation Loss: 66.71
Epoch 16, Validation Loss: 72.49
Epoch 17, Validation Loss: 70.17
Epoch 18, Validation Loss: 69.57
Epoch 19, Validation Loss: 73.20
Epoch 20, Validation Loss: 68.68
Epoch 21, Validation Loss: 67.91
Epoch 22, Validation Loss: 67.78
Epoch 23, Validation Loss: 73.36
Epoch 24, Validation Loss: 62.07
Epoch 25, Validation Loss: 65.80
Epoch 26, Validation Loss: 61.08
Epoch 27, Validation Loss: 77.95
Epoch 28, Validation Loss: 63.95
Epoch 29, Validation Loss: 67.13
Epoch 30, Valida

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_dev, a_dev))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lt_50-10.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:06<00:00, 162.32it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean     60.863243               0.0      0.003321
std      18.257178               0.0      0.001077
min      22.440795               0.0      0.001000
25%      48.507281               0.0      0.002512
50%      58.533421               0.0      0.003002
75%      71.119708               0.0      0.004000
max     157.461830               0.0      0.008532
Number of infeasible solution: 0
